In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
import pickle,os,shutil
torch.manual_seed(1)

# What is a Gradient descent?
참고
- http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html


# 0. 텐서보드

In [2]:
port= '6006'
try:
    shutil.rmtree('runs/')
except:
    pass

# 1. 데이터

In [3]:
BATCH_SIZE= 64

train_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=True) # 이동평균이 튀는걸 방지

test_dataset = vdatasets.MNIST(root='../data/MNIST/',
                               train=False, 
                               transform=transforms.ToTensor(),
                               download=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True,
                                           num_workers=2)

# 2. model

In [4]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.linear1= nn.Linear(input_size, hidden_size)
        self.linear2= nn.Linear(hidden_size, hidden_size)
        self.linear3= nn.Linear(hidden_size, output_size)
        
        # In : {배치사이즈, 차원수} => Out : (배치사이즈, 차원수)
        self.bn1= nn.BatchNorm1d(hidden_size)
        self.bn2= nn.BatchNorm1d(hidden_size)
    def forward(self, inputs):
        outputs= self.bn1(self.linear1(inputs))
        outputs= F.relu(outputs)
        outputs= self.bn2(self.linear2(outputs))
        outputs= F.relu(outputs)
        return self.linear3(outputs)

In [5]:
EPOCH=5
LR=0.1

INPUT_SIZE= 784
HIDDEN_SIZE= 512
OUTPUT_SIZE=10

model=NN(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
loss_function= nn.CrossEntropyLoss()

# Optimizer

In [6]:
sgd= optim.SGD(model.parameters(), lr=LR)

sgd_momentum = optim.SGD(model.parameters(),lr=LR,momentum=0.9)
sgd_nesterov_momentum = optim.SGD(model.parameters(),lr=LR,momentum=0.9,nesterov=True)

adagrad = optim.Adagrad(model.parameters(),lr=LR)
rmsprop = optim.RMSprop(model.parameters(),lr=LR,alpha=0.99)
adam = optim.Adam(model.parameters(),lr=LR,betas=(0.9,0.999))

In [22]:
a= sgd.a

In [19]:
str(a)

"<class 'torch.optim.sgd.SGD'>"

# 3. 학습

In [24]:
EPOCH=5
LR=0.1
HIDDEN_SIZE = 512
OPTIM_OPS = ['sgd', 'sgd_momentum', 'sgd_nesterov_momentum', 'adagrad',
            'rmsprop', 'adam'] 

for op_name in OPTIM_OPS:
    writer = SummaryWriter(comment="-"+op_name)
    
    model = NN(784, HIDDEN_SIZE, 10)
    loss_function = nn.CrossEntropyLoss()
    
    exec('optimizer= {}'.format(op_name))
    
    # 트레이닝
    print(op_name + " training start!")
    for epoch in range(EPOCH):
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = Variable(inputs).view(-1,784), Variable(targets)
            model.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            
            writer.add_scalars('data/optimizer/',{op_name : loss.data[0]},(i+1)+(epoch*len(train_loader)))
    print("done")
writer.close()

sgd training start!


/Users/a408109/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


done
sgd_momentum training start!
done
sgd_nesterov_momentum training start!
done
adagrad training start!
done
rmsprop training start!
done
adam training start!
done


In [25]:
port

'6006'

In [ ]:
!tensorboard --logdir runs --port 6006

/Users/a408109/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.9.0 at http://408109-HC-D16A57:6006 (Press CTRL+C to quit)
